In [25]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm

class DividendInvestmentModel:
    def __init__(self, first_investment, periodic_investment, forecast_period,
                 dividend_yield_mean, dividend_yield_std, dividend_frequency,
                 stock_growth_mean, stock_growth_std, sell_frequency,
                 sell_percentage, reinvest_percentage):
        self.first_investment = first_investment
        self.periodic_investment = periodic_investment
        self.forecast_period = forecast_period  # in months
        self.dividend_yield_mean = dividend_yield_mean
        self.dividend_yield_std = dividend_yield_std
        self.dividend_frequency = dividend_frequency.lower()  # 'monthly', 'quarterly', 'annually'
        self.sell_frequency = sell_frequency.lower()  # 'monthly', 'quarterly', 'annually'
        self.stock_growth_mean = stock_growth_mean
        self.stock_growth_std = stock_growth_std
        self.sell_percentage = sell_percentage / 100  # Convert to decimal
        self.reinvest_percentage = reinvest_percentage / 100  # Convert to decimal
        self.dividend_payment_intervals = {'monthly': 1, 'quarterly': 3, 'annually': 12}
        self.append_investment = False

    def _get_bounded_dividend_yield(self):
        """
        Generate a dividend yield randomly from a truncated normal distribution.
        Bounds: 0% to reasonable upper bound (e.g., 20%).
        """
        lower_bound, upper_bound = 0, 0.20  # 0% to 20%
        a, b = (lower_bound - self.dividend_yield_mean) / self.dividend_yield_std, \
               (upper_bound - self.dividend_yield_mean) / self.dividend_yield_std
        return truncnorm.rvs(a, b, loc=self.dividend_yield_mean, scale=self.dividend_yield_std)

    def _get_stock_growth_rate(self):
        """
        Generate a stock price growth rate from a normal distribution.
        """
        return np.random.normal(self.stock_growth_mean, self.stock_growth_std)

    def generate_forecast(self):
        """
        Generate a DataFrame of investment details including dividends, stock appreciation, and sell logic.
        """
        # Initialize columns
        periods = []
        investment_amounts = self.periodic_investment if isinstance(self.periodic_investment, list) else []
        if investment_amounts == []:
            self.append_investment = True
        total_investments = []
        stock_values = []
        dividend_yields = []
        dividends = []
        sell_proceeds = []
        capital_gains = []
        reinvested_amounts = []
        cumulative_reset_flags = []

        # Initialize running totals
        current_date = self.first_investment
        cumulative_investment = 0
        current_stock_value = 0

        sell_interval = self.dividend_payment_intervals[self.sell_frequency]
        dividend_interval = self.dividend_payment_intervals[self.dividend_frequency]

        # Loop through each month in the forecast period
        for period in range(1, self.forecast_period + 1):
            # Add to period
            periods.append(current_date)
            
            # Add periodic investment
            if self.append_investment:
                investment_amounts.append(self.periodic_investment) 
                cumulative_investment += self.periodic_investment
                current_stock_value += self.periodic_investment
            
            else:
                cumulative_investment +- investment_amounts[period-1]  
                current_stock_value += investment_amounts[period-1]
            
            total_investments.append(cumulative_investment)
            

            # Apply stock price growth
            stock_growth_rate = self._get_stock_growth_rate()
            current_stock_value *= (1 + stock_growth_rate)
            stock_values.append(current_stock_value)

            # Handle dividend payments based on frequency
            if period % dividend_interval == 0:
                dividend_yield = self._get_bounded_dividend_yield()
                dividend = current_stock_value * dividend_yield
            else:
                dividend_yield = np.nan
                dividend = np.nan
            dividend_yields.append(dividend_yield)
            dividends.append(dividend)

            # Handle selling logic based on frequency
            if period % sell_interval == 0:
                sell_amount = current_stock_value * self.sell_percentage
                invested_portion_sold = cumulative_investment * self.sell_percentage
                capital_gain = sell_amount - invested_portion_sold

                # Reinvestment logic
                reinvest_amount = sell_amount * self.reinvest_percentage
                cumulative_investment = reinvest_amount  # Reset cumulative investment to reinvested amount
                current_stock_value = reinvest_amount  # Reset stock value

                # Record values
                sell_proceeds.append(sell_amount)
                capital_gains.append(capital_gain)
                reinvested_amounts.append(reinvest_amount)
                cumulative_reset_flags.append(1)
            else:
                sell_proceeds.append(np.nan)
                capital_gains.append(np.nan)
                reinvested_amounts.append(np.nan)
                cumulative_reset_flags.append(0)

            # Increment date by one month
            current_date += pd.DateOffset(months=1)

        # Create DataFrame
        forecast_df = pd.DataFrame({
            'Period': periods,
            'Investment Amount': investment_amounts,
            'Total Investment': np.round(total_investments,2),
            'Stock Value': np.round(stock_values,2),
            'Sell Proceeds': np.round(sell_proceeds,2),
            'Capital Gain': np.round(capital_gains,2),
            'Reinvested Amount': np.round(reinvested_amounts,2),
            'Dividend Yield': dividend_yields,
            'Dividend': np.round(dividends,2),
            'Cumulative Reset': cumulative_reset_flags
        })
        
        forecast_df['Period'] = pd.to_datetime(forecast_df['Period']).dt.strftime('%Y-%m-%d')

        return forecast_df
    
    def transform_to_cashflow(self, forecast_df):
        """
        Transform the forecast DataFrame into a cashflow format with Revenue and Expense.
        Revenue = Dividend + Sell Proceeds
        Expense = Investment Amount + Reinvested Amount
        """
        cashflow_df = pd.DataFrame({
            'Period': forecast_df['Period'],
            'Revenue': forecast_df[['Dividend', 'Sell Proceeds']].sum(axis=1, skipna=True),
            'Expense': forecast_df[['Investment Amount', 'Reinvested Amount']].sum(axis=1, skipna=True)
        })
        return cashflow_df





In [26]:
# open json file
import json
with open('without_improvement_profit.json') as f:
    data = json.load(f)
    
data

[9522.009333333333,
 5707.822666666669,
 6875.468,
 7381.0266666666685,
 8157.894666666666,
 7479.442666666666,
 5929.876,
 8369.243999999999,
 6401.812000000002,
 7955.712000000001,
 7152.488000000001,
 4984.169333333333]

In [27]:

model = DividendInvestmentModel(
    first_investment=pd.Timestamp("2024-01-01"),
    periodic_investment=data,  # Monthly investment
    forecast_period=12,  # 12 months
    dividend_yield_mean=0.05,  # 5% mean dividend yield
    dividend_yield_std=0.01,  # 1% standard deviation
    dividend_frequency='quarterly',  # Dividends paid quarterly
    stock_growth_mean=0.06,  # 6% annual stock growth
    stock_growth_std=0.02,  # 2% stock growth volatility
    sell_frequency='annually',  # Sell stocks annually
    sell_percentage=100,  # Sell 100% of stock value
    reinvest_percentage=50  # Reinvest 50% of the proceeds
)

forecast_df = model.generate_forecast()
forecast_df

,Period,Investment Amount,Total Investment,Stock Value,Sell Proceeds,Capital Gain,Reinvested Amount,Dividend Yield,Dividend,Cumulative Reset
0,2024-01-01,9522.009333,0,10114.00,NaN,NaN,NaN,NaN,NaN,0
1,2024-02-01,5707.822667,0,16597.30,NaN,NaN,NaN,NaN,NaN,0
2,2024-03-01,6875.468000,0,24882.20,NaN,NaN,NaN,0.047927,1192.54,0
3,2024-04-01,7381.026667,0,34504.72,NaN,NaN,NaN,NaN,NaN,0
4,2024-05-01,8157.894667,0,45528.60,NaN,NaN,NaN,NaN,NaN,0
5,2024-06-01,7479.442667,0,55032.55,NaN,NaN,NaN,0.056450,3106.59,0
6,2024-07-01,5929.876000,0,66411.37,NaN,NaN,NaN,NaN,NaN,0
7,2024-08-01,8369.244000,0,77871.90,NaN,NaN,NaN,NaN,NaN,0
8,2024-09-01,6401.812000,0,89128.91,NaN,NaN,NaN,0.051054,4550.42,0
9,2024-10-01,7955.712000,0,101043.79,NaN,NaN,NaN,NaN,NaN,0


In [10]:
a = forecast_df['Investment Amount'].values

# check if a is an array resulting in true or false
b = 2 if isinstance(a, np.ndarray) else 0
b


2

In [15]:
# check if data is a list resulting in true or false
b = 2 if isinstance(data, list) else 0
b

2

In [31]:
# multiply data in the list by 2
data = [i*2 for i in data]
data

[19044.018666666667,
 11415.645333333337,
 13750.936,
 14762.053333333337,
 16315.789333333332,
 14958.885333333332,
 11859.752,
 16738.487999999998,
 12803.624000000003,
 15911.424000000003,
 14304.976000000002,
 9968.338666666667]